In [89]:
import pandas as pd
import numpy as np

In [120]:
detail = pd.read_excel('PBToulouse2022/idvotant-idprojet.xlsx')
election_agg = pd.read_csv('projectsAgg.csv', sep = ";")

## cleaned voter file
detail = detail[detail.vote_finished==True]
detail = detail[detail.checked_out_at < '2022-11-01']
detail.project_id = detail.project_id.astype('int')
detail = detail[detail.project_id.isin(election_agg.project_id.tolist())]

In [121]:
election_agg

,project_id,project_name,category,cost,district,votes
0,116,Création d’abris pour la faune,Nature en ville,4000.0,10 - Jolimont / Soupetard / Roseraie / Gloire ...,38
1,180,Pépinière-jardin du quartier Reynerie-Bellefon...,Nature en ville,75000.0,17 - Mirail-Université / Reynerie / Bellefontaine,20
2,124,Toilettes publiques Bois de Limayrac,Cadre de vie,60000.0,11 - Bonhoure / Guilheméry / Château de l'Hers...,23
3,13,Esthétique des conteneurs de recyclage,Cadre de vie,4000.0,1 - Capitole / Arnaud Bernard / Carmes,14
4,37,Clarifier les priorités à la sortie de la rue ...,Cadre de vie,40000.0,6 - Saint-Cyprien,9
...,...,...,...,...,...,...
195,90,Installer un filtre à pollution Place Job,Nature en ville,80000.0,7 - Sept Deniers / Ginestous-Sesquières / Lalande,24
196,149,Un jardin en mouvement le long de la voie ferr...,Nature en ville,11000.0,5 - Saint-Michel / Saint-Agne / Empalot / Le B...,50
197,113,Personne en situation de handicap,Cadre de vie,200000.0,10 - Jolimont / Soupetard / Roseraie / Gloire ...,40
198,143,Lampadaires à détecteur de mouvement piste cyc...,Énergie,50000.0,13 - Rangueil / Sauzelong / Jules-Julien / Pec...,50


In [122]:
# create a projects list:
project_list = []
for _, row in election_agg.iterrows():
    dictionary = {}
    for column in election_agg.columns:
        dictionary[column] = row[column]
    project_list.append(dictionary)

print(project_list)

[{'project_id': 116, 'project_name': 'Création d’abris pour la faune', 'category': 'Nature en ville', 'cost': 4000.0, 'district': '10 - Jolimont / Soupetard / Roseraie / Gloire / Gramont / Amouroux', 'votes': 38}, {'project_id': 180, 'project_name': 'Pépinière-jardin du quartier Reynerie-Bellefontaine', 'category': 'Nature en ville', 'cost': 75000.0, 'district': '17 - Mirail-Université / Reynerie / Bellefontaine', 'votes': 20}, {'project_id': 124, 'project_name': 'Toilettes publiques Bois de Limayrac', 'category': 'Cadre de vie', 'cost': 60000.0, 'district': "11 - Bonhoure / Guilheméry / Château de l'Hers / Limayrac / Côte Pavée", 'votes': 23}, {'project_id': 13, 'project_name': 'Esthétique des conteneurs de recyclage', 'category': 'Cadre de vie', 'cost': 4000.0, 'district': '1 - Capitole / Arnaud Bernard / Carmes', 'votes': 14}, {'project_id': 37, 'project_name': 'Clarifier les priorités à la sortie de la rue Réclusane', 'category': 'Cadre de vie', 'cost': 40000.0, 'district': '6 - Sa

In [123]:
# set of districts
districts = [d for d in set(election_agg.district.tolist())]

In [124]:
def greedy_assignment(project_list, budget=400000):
    winners = []
    while budget > 0 and len(project_list) > 0:
        w = project_list.pop(0)
        if budget - w['cost'] >= 0:
            winners.append(w)
            budget = budget - w['cost']
        
    return winners

In [ ]:
def greedy_assignment_by_districts(project_list, districts, budget_by_district = 400000):
    result = dict()
    for d in districts:
        budget = budget_by_district
        projects = [p for p in project_list if p['district'] == d]
        _sorted = sorted(projects, key=lambda x: x["votes"], reverse=True)
        winners = []
        while budget > 0 and len(_sorted) > 0:
            w = _sorted.pop(0)
            if budget - w['cost'] >= 0:
                winners.append(w)
                budget = budget - w['cost']
        result[d] = winners
    return result

In [128]:
# 1. list of projects
p = [p for p in project_list]
p_sorted = sorted(p, key=lambda x: x["votes"], reverse=True)

winners = greedy_assignment_by_districts(project_list=p_sorted, districts=districts)

In [129]:
for k,v in winners.items():
    print(f'nbhood-{k.split("-")[0].strip()}:', f"{len(v)} projects, total: {np.sum([p['cost'] for p in v])}")

nbhood-7: 6 projects, total: 397300.0
nbhood-11: 5 projects, total: 398500.0
nbhood-8: 6 projects, total: 393000.0
nbhood-5: 5 projects, total: 400000.0
nbhood-12: 6 projects, total: 385000.0
nbhood-19: 4 projects, total: 390000.0
nbhood-13: 5 projects, total: 396200.0
nbhood-17: 2 projects, total: 400000.0
nbhood-9: 9 projects, total: 362700.0
nbhood-3: 5 projects, total: 350700.0
nbhood-6: 4 projects, total: 399000.0
nbhood-20: 3 projects, total: 400000.0
nbhood-2: 8 projects, total: 396900.0
nbhood-1: 6 projects, total: 396000.0
nbhood-15: 7 projects, total: 398200.0
nbhood-14: 7 projects, total: 397200.0
nbhood-10: 5 projects, total: 400000.0
nbhood-16: 6 projects, total: 375000.0
nbhood-4: 8 projects, total: 320320.0
nbhood-18: 6 projects, total: 377700.0


In [134]:
winners["7 - Sept Deniers / Ginestous-Sesquières / Lalande"]

[{'project_id': 84,
  'project_name': "Composte partagé 'lalala'lande",
  'category': 'Déchets et Recyclage',
  'cost': 7000.0,
  'district': '7 - Sept Deniers / Ginestous-Sesquières / Lalande',
  'votes': 61},
 {'project_id': 86,
  'project_name': 'Zone verte des Ponts-Jumeaux : équipements sportifs',
  'category': 'Cadre de vie',
  'cost': 150000.0,
  'district': '7 - Sept Deniers / Ginestous-Sesquières / Lalande',
  'votes': 59},
 {'project_id': 87,
  'project_name': 'Bataille Militaire aux 7 deniers en 1814 (création d’une statue de soldat)',
  'category': 'Cadre de vie',
  'cost': 60000.0,
  'district': '7 - Sept Deniers / Ginestous-Sesquières / Lalande',
  'votes': 52},
 {'project_id': 83,
  'project_name': "Aire sportive et ludique intergénérationnelle autour d'un pumptrack",
  'category': 'Cadre de vie',
  'cost': 150000.0,
  'district': '7 - Sept Deniers / Ginestous-Sesquières / Lalande',
  'votes': 26},
 {'project_id': 81,
  'project_name': 'Aménager le Parc des Sept-Deniers 